In [1]:
import numpy as np
import sys
from pyinstrument import Profiler
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import os
sys.path.append(os.path.dirname(os.getcwd()))
from bart_playground import *

import bartz

In [2]:
proposal_probs = {"grow" : 0.5,
                  "prune" : 0.5}
generator = DataGenerator(n_samples=160, n_features=2, noise=0.1, random_seed=42)
X, y = generator.generate(scenario="piecewise_flat")
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
np.set_printoptions(suppress=True)
print(y_train[:12])

[ 0.50327821  0.60672224  0.26898966  0.55211673  0.50693811  0.66162097
 -0.64127659  0.65112284  0.03487759  0.23276531  0.44055996  0.38216964]


In [3]:
profiler = Profiler()
profiler.start()
bart = ChangeNumTreeBART(ndpost=200, nskip=100, n_trees=200, proposal_probs=proposal_probs)
bart.fit(X_train, y_train)
profiler.stop()
profiler.print()

Iterations:   2%|▏         | 6/300 [00:06<03:42,  1.32it/s]

Total ratio: [10525.38527597]
Transition ratio: 399.9999999999999
Total ratio: [18.51929284]
Transition ratio: 132.66666666666663
Total ratio: [66.26559397]
Transition ratio: 98.99999999999999
Total ratio: [39.84843089]
Transition ratio: 197.0


Iterations:   7%|▋         | 20/300 [00:06<00:34,  8.02it/s]

Total ratio: [11.67565589]
Transition ratio: 195.99999999999991
Total ratio: [9.89676041]
Transition ratio: 156.0
Total ratio: [589.31888641]
Transition ratio: 194.00000000000003
Total ratio: [66.83246486]
Transition ratio: 193.0
Total ratio: [16.99472358]
Transition ratio: 192.0


Iterations:   8%|▊         | 24/300 [00:06<00:25, 10.78it/s]

...

Iterations:  95%|█████████▌| 286/300 [00:13<00:00, 46.05it/s]

Total ratio: [3.54074951]
Transition ratio: 90.0
Total ratio: [3.57685651]
Transition ratio: 44.50000000000001
Total ratio: [2.17918436]
Transition ratio: 66.00000000000003
Total ratio: [91.94510831]
Transition ratio: 43.49999999999999
Total ratio: [47.42235083]
Transition ratio: 85.99999999999997


Iterations: 100%|██████████| 300/300 [00:13<00:00, 21.43it/s]

Total ratio: [6.59715246]
Transition ratio: 56.666666666666664
Total ratio: [14.24466157]
Transition ratio: 83.99999999999999
Total ratio: [22.73037221]
Transition ratio: 83.00000000000003



  _     ._   __/__   _ _  _  _ _/_   Recorded: 00:10:01  Samples:  11913
 /_//_/// /_\ / //_// / //_'/ //     Duration: 14.007    CPU time: 2.500
/   _/                      v5.0.1

Profile at C:\Users\ztykk\AppData\Local\Temp\ipykernel_27336\4120808940.py:2

14.007 MainThread  <thread>:18216
├─ 13.461 _run_module_as_main  <frozen runpy>:173
│     [19 frames hidden]  <frozen runpy>, ipykernel_launcher, t...
│        13.461 ZMQInteractiveShell.run_ast_nodes  IPython\core\interactiveshell.py:3420
│        └─ 13.461 <module>  ..\..\..\Temp\ipykernel_27336\4120808940.py:1
│           └─ 13.461 ChangeNumTreeBART.fit  bart_playground\bart.py:25
│              └─ 13.461 NTreeSampler.run  bart_playground\samplers.py:73
│                 └─ 13.356 NTreeSampler.one_iter  bart_playground\samplers.py:292
│                    ├─ 5.447 NTreeSampler.log_mh_ratio  bart_playground\samplers.py:279
│                    │  ├─ 3.955 BARTLikelihood.trees_log_marginal_lkhd_ratio  bart_playground\priors.py:3

In [4]:
bart.trace[-1].n_trees

82

In [5]:
rf = RandomForestRegressor()
lr = LinearRegression()
rf.fit(X_train, y_train)
lr.fit(X_train, y_train)

btz = bartz.BART.gbart(np.transpose(X_train), y_train, ntree=100, ndpost=200, nskip=100)
btpred_all = btz.predict(np.transpose(X_test))
btpred = np.mean(np.array(btpred_all), axis=0)

Iteration 100/300 P_grow=0.55 P_prune=0.45 A_grow=0.36 A_prune=0.36 (burnin)
Iteration 200/300 P_grow=0.57 P_prune=0.43 A_grow=0.35 A_prune=0.37
Iteration 300/300 P_grow=0.57 P_prune=0.43 A_grow=0.39 A_prune=0.40


In [6]:
bart.predict(X_test)

array([ 0.0952605 , -0.32599776, -0.05692736,  0.02309529,  0.22419512,
       -0.33314667, -0.02604076,  0.27238211,  0.1039618 ,  0.17654851,
        0.23924069,  0.3149124 ,  0.13136614,  0.13122453, -0.00256791,
        0.15298233,  0.05632352,  0.11565711,  0.1813425 ,  0.13353535,
        0.10268533,  0.08066375,  0.10317648,  0.09998106,  0.25897948,
        0.19059288, -0.22947472,  0.23815233, -0.12711846,  0.1069246 ,
        0.08235778,  0.18442195,  0.16016996,  0.02879949,  0.29804783,
        0.16681494,  0.28823319, -0.36815578,  0.02575707,  0.07557683])

In [7]:

models = {"bart" : bart, 
          "rf" : rf, 
          "lr" : lr,
          "btz" : btz}
results = {}
for model_name, model in models.items():
    if model_name == "btz":
        results[model_name] = mean_squared_error(y_test, btpred)
    else:
        results[model_name] = mean_squared_error(y_test, model.predict(X_test))
results

{'bart': 0.09076278893774695,
 'rf': 0.020135989144067713,
 'lr': 0.048045521328019404,
 'btz': 0.02328283761397566}